<a href="https://colab.research.google.com/github/kundajelab/revcomp_experiments/blob/master/CTCG_RegressionExample_Standard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# #We want to prepare a bed file that has +/- 1kb around the summit, followed by
# # the signal strength
# ! zcat peaks_with_signal.bed.gz | perl -lane 'print $F[0]."\t".($F[1]+$F[9])."\t".($F[1]+$F[9])."\t+\t".($F[6])' | egrep -w 'chr1|chr2|chr3|chr4|chr5|chr6|chr7|chr8|chr9|chr10|chr11|chr12|chr13|chr14|chr15|chr16|chr17|chr18|chr19|chr20|chr21|chr22|chrX|chrY' | gzip -c > summits_with_signal.bed.gz

# #We split into training/test/validation set by chromosome
# !zcat summits_with_signal.bed.gz | egrep -w 'chr1|chr8|chr21' | gzip -c > test_summits_with_signal.bed.gz
# !zcat summits_with_signal.bed.gz | egrep -w 'chr22' | gzip -c > valid_summits_with_signal.bed.gz
# !zcat summits_with_signal.bed.gz | egrep -w -v 'chr1|chr8|chr21|chr22' | gzip -c > train_summits_with_signal.bed.gz

In [1]:
from seqdataloader.batchproducers import coordbased
import gzip
import numpy as np

class ColsInBedFile(
    coordbased.coordstovals.core.AbstractSingleNdarrayCoordsToVals):
    def __init__(self, gzipped_bed_file, **kwargs):
        super(ColsInBedFile, self).__init__(**kwargs)
        self.gzipped_bed_file = gzipped_bed_file
        coords_to_vals = {}
        for row in gzip.open(gzipped_bed_file, 'rb'):
            row = row.decode("utf-8").rstrip()
            split_row = row.split("\t")
            chrom_start_end = split_row[0]+":"+split_row[1]+"-"+split_row[2]
            vals = np.array([float(x) for x in split_row[4:]])
            coords_to_vals[chrom_start_end] = vals
        self.coords_to_vals = coords_to_vals
        
    def _get_ndarray(self, coors):
        to_return = []
        for coor in coors:
            chrom_start_end = (coor.chrom+":"
                               +str(coor.start)+"-"+str(coor.end))
            to_return.append(self.coords_to_vals[chrom_start_end])
        return np.array(to_return)
    
    
inputs_coordstovals = coordbased.coordstovals.fasta.PyfaidxCoordsToVals(
  genome_fasta_path= '/mnt/data/annotations/by_release/hg38/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta',
  center_size_to_use=1000)

targets_coordstovals = ColsInBedFile(
       gzipped_bed_file="summits_with_signal.bed.gz")
            
keras_train_batch_generator = coordbased.core.KerasBatchGenerator(
    coordsbatch_producer=coordbased.coordbatchproducers.SimpleCoordsBatchProducer(
      bed_file="train_summits_with_signal.bed.gz",
      #coord_batch_transformer=coordbased.coordbatchtransformers.ReverseComplementAugmenter(),
      batch_size=64,
      shuffle_before_epoch=True,
      seed=1234
    ),
    inputs_coordstovals=inputs_coordstovals,
    targets_coordstovals=targets_coordstovals
)


keras_valid_batch_generator = coordbased.core.KerasBatchGenerator(
    coordsbatch_producer = coordbased.coordbatchproducers.SimpleCoordsBatchProducer(
        bed_file="valid_summits_with_signal.bed.gz", 
        batch_size=64, 
        shuffle_before_epoch=True, 
        seed=1234
    ),
    inputs_coordstovals=inputs_coordstovals, 
    targets_coordstovals=targets_coordstovals
)

keras_test_batch_generator = coordbased.core.KerasBatchGenerator(
    coordsbatch_producer = coordbased.coordbatchproducers.SimpleCoordsBatchProducer(
        bed_file="test_summits_with_signal.bed.gz", 
        batch_size = 64, 
        shuffle_before_epoch = True, 
        seed = 1234
    ), 
    inputs_coordstovals = inputs_coordstovals, 
    targets_coordstovals = targets_coordstovals
)


keras_train_batch_generator_augment = coordbased.core.KerasBatchGenerator(
    coordsbatch_producer=coordbased.coordbatchproducers.SimpleCoordsBatchProducer(
      bed_file="train_summits_with_signal.bed.gz",
      coord_batch_transformer=coordbased.coordbatchtransformers.ReverseComplementAugmenter(),
      batch_size=128,
      shuffle_before_epoch=True,
      seed=1234
    ),
    inputs_coordstovals=inputs_coordstovals,
    targets_coordstovals=targets_coordstovals
)


keras_valid_batch_generator_augment = coordbased.core.KerasBatchGenerator(
    coordsbatch_producer = coordbased.coordbatchproducers.SimpleCoordsBatchProducer(
        bed_file="valid_summits_with_signal.bed.gz",
        coord_batch_transformer=coordbased.coordbatchtransformers.ReverseComplementAugmenter(),
        batch_size=128, 
        shuffle_before_epoch=True, 
        seed=1234
    ),
    inputs_coordstovals=inputs_coordstovals, 
    targets_coordstovals=targets_coordstovals
)

keras_test_batch_generator_augment = coordbased.core.KerasBatchGenerator(
    coordsbatch_producer = coordbased.coordbatchproducers.SimpleCoordsBatchProducer(
        bed_file="test_summits_with_signal.bed.gz",
        coord_batch_transformer=coordbased.coordbatchtransformers.ReverseComplementAugmenter(),
        batch_size = 128, 
        shuffle_before_epoch = True, 
        seed = 1234
    ), 
    inputs_coordstovals = inputs_coordstovals, 
    targets_coordstovals = targets_coordstovals
)

Using TensorFlow backend.


In [2]:
y_test = np.array([val for batch in keras_test_batch_generator for val in batch[1]], dtype = 'float32') 

In [3]:
# !pip install git+https://github.com/kundajelab/simdna.git@v0.4.3.1#egg=simdna
# !git clone https://github.com/kundajelab/revcomp_experiments.git
# %cd revcomp_experiments/
# !python setup.py install
# %cd ..

In [4]:
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import random_ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.ops import array_ops
from tensorflow.python.framework import ops
import numbers
from tensorflow.python.framework import tensor_util
def _get_noise_shape(x, noise_shape):
  # If noise_shape is none return immediately.
  if noise_shape is None:
    return array_ops.shape(x)

  try:
    # Best effort to figure out the intended shape.
    # If not possible, let the op to handle it.
    # In eager mode exception will show up.
    noise_shape_ = tensor_shape.as_shape(noise_shape)
  except (TypeError, ValueError):
    return noise_shape

  if x.shape.dims is not None and len(x.shape.dims) == len(noise_shape_.dims):
    new_dims = []
    for i, dim in enumerate(x.shape.dims):
      if noise_shape_.dims[i].value is None and dim.value is not None:
        new_dims.append(dim.value)
      else:
        new_dims.append(noise_shape_.dims[i].value)
    return tensor_shape.TensorShape(new_dims)

  return noise_shape

class MCRCDropout(Layer):
    """Applies MC Dropout to the input.
       The applied noise vector is symmetric to reverse complement symmetry
       Class structure only slightly adapted 
    Dropout consists in randomly setting
    a fraction `rate` of input units to 0 at each update during training time,
    which helps prevent overfitting.
    Remains active ative at test time so sampling is required
    # Arguments
        rate: float between 0 and 1. Fraction of the input units to drop.
        noise_shape: 1D integer tensor representing the shape of the
            binary dropout mask that will be multiplied with the input.
            For instance, if your inputs have shape
            `(batch_size, timesteps, features)` and
            you want the dropout mask to be the same for all timesteps,
            you can use `noise_shape=(batch_size, 1, features)`.
        seed: A Python integer to use as random seed.
    # References
        - [Dropout: A Simple Way to Prevent Neural Networks from Overfitting](http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf)
    """
    def __init__(self, rate, noise_shape=None, seed=None, **kwargs):
        super(MCRCDropout, self).__init__(**kwargs)
        self.rate = min(1., max(0., rate))
        self.noise_shape = noise_shape
        self.seed = seed
        self.supports_masking = True
        
    def build(self, input_shape):
        self.num_input_chan = input_shape[2]
        super(MCRCDropout, self).build(input_shape)

    def _get_noise_shape(self, inputs):
        if self.noise_shape is None:
            return self.noise_shape

        symbolic_shape = K.shape(inputs)
        noise_shape = [symbolic_shape[axis] if shape is None else shape
                       for axis, shape in enumerate(self.noise_shape)]
        return tuple(noise_shape)

    def call(self, inputs, training=None):
        if 0. < self.rate < 1.:
            import numpy as np
            noise_shape = self._get_noise_shape(inputs)
            x = inputs
            seed = self.seed
            keep_prob = 1. - self.rate
            if seed is None:
                seed = np.random.randint(10e6)
            # the dummy 1. works around a TF bug
            # (float32_ref vs. float32 incompatibility)
            x= x*1
            name = None
            with ops.name_scope(name, "dropout", [x]) as name:
                x = ops.convert_to_tensor(x, name="x")
                if not x.dtype.is_floating:
                    raise ValueError("x has to be a floating point tensor since it's going to"
                       " be scaled. Got a %s tensor instead." % x.dtype)
                if isinstance(keep_prob, numbers.Real) and not 0 < keep_prob <= 1:
                    raise ValueError("keep_prob must be a scalar tensor or a float in the "
                       "range (0, 1], got %g" % keep_prob)
                keep_prob = ops.convert_to_tensor(
                             keep_prob, dtype=x.dtype, name="keep_prob")
                keep_prob.get_shape().assert_is_compatible_with(tensor_shape.scalar())

                # Do nothing if we know keep_prob == 1
                if tensor_util.constant_value(keep_prob) == 1:
                    return x

                noise_shape = _get_noise_shape(x, noise_shape)
                # uniform [keep_prob, 1.0 + keep_prob)
                random_tensor = keep_prob
                random_tensor += random_ops.random_uniform(
                noise_shape, seed=seed, dtype=x.dtype)
               
                # 0. if [keep_prob, 1.0) and 1. if [1.0, 1.0 + keep_prob)
                binary_tensor = math_ops.floor(random_tensor)
                dim = binary_tensor.shape[2]//2

                symmetric_binary = K.concatenate(
                    tensors = [
                      binary_tensor[:,:,int(self.num_input_chan/2):], 
                      binary_tensor[:,:,int(self.num_input_chan/2):][::,::-1,::-1]], 
                  axis=2)
                ret = math_ops.div(x, keep_prob) * symmetric_binary
                
                return ret


    def get_config(self):
        config = {'rate': self.rate,
                  'noise_shape': self.noise_shape,
                  'seed': self.seed}
        base_config = super(MCRCDropout, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

NameError: name 'Layer' is not defined

In [ ]:
class RevCompSpatialDropout1D(Dropout): 
    def __init__(self, rate,**kwargs): 
        super(RevCompSpatialDropout1D, self).__init__(rate, **kwargs)
        self.seed = 3
        self.input_spec = InputSpec(ndim = 3)

    def _get_noise_shape(self, inputs): 
        input_shape = K.shape(inputs)
        noise_shape = (input_shape[0], 1, 1, int(self.num_input_chan/2)) 
        return noise_shape
        
    def build(self, input_shape):
        self.num_input_chan = input_shape[2]
        self.input_len = input_shape[1]
        super(RevCompSpatialDropout1D, self).build(input_shape)

    def call(self, inputs, training=None): 
        inputs_fwdandrevconcat = K.concatenate(
                tensors = [
                    inputs[:,:,None,:int(self.num_input_chan/2)],
                    inputs[:,:,None,int(self.num_input_chan/2):][:,:,:,::-1]],
                axis=2)

        if 0. < self.rate < 1.: 
            noise_shape = self._get_noise_shape(inputs)
            def dropped_inputs(): 
                dropped = K.dropout(inputs_fwdandrevconcat,
                                    self.rate, noise_shape, seed = self.seed)
                dropped = K.reshape(dropped, (-1, int(self.input_len), int(self.num_input_chan)))
                return K.concatenate(
                    tensors = [
                        dropped[:,:,:int(self.num_input_chan/2)],
                        dropped[:,:,int(self.num_input_chan/2):][:,:,::-1]],
                    axis=-1)

            return K.in_train_phase(dropped_inputs, inputs, training = training)

        return inputs

In [ ]:
class RevCompSumPool(Layer): 
    def __init__(self, **kwargs): 
        super(RevCompSumPool, self).__init__(**kwargs)

    def build(self, input_shape):
        self.num_input_chan = input_shape[2]
        super(RevCompSumPool, self).build(input_shape)

    def call(self, inputs): 
        #divide by sqrt 2 for variance preservation
        inputs = (inputs[:,:,:int(self.num_input_chan/2)] + inputs[:,:,int(self.num_input_chan/2):][:,::-1,::-1])/(1.41421356237)
        return inputs
      
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], int(input_shape[2]/2))

In [ ]:
import keras 
import keras_genomics
import numpy as np
import keras.layers as k1
import simdna

from keras import backend as K 
from keras.layers.core import Dropout 
from keras.layers.core import Flatten
from keras.layers import Input
from keras.engine import Layer
from keras.models import Sequential 
from keras.engine.base_layer import InputSpec
from keras.models import Model
from keras.models import load_model

In [ ]:
kernel_size = 15
filters= 15
input_length = 1000

from numpy.random import seed
from tensorflow import set_random_seed
from keras.callbacks import EarlyStopping, History, ModelCheckpoint

seed_num = 1000
seed(seed_num)
set_random_seed(seed_num)

In [ ]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

In [25]:
scale = 1.0

reg_model = keras.models.Sequential()
reg_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        input_shape=keras_train_batch_generator[0][0].shape[1:],
                        padding="same"))
reg_model.add(k1.core.Activation("relu"))
reg_model.add(k1.Dropout(0.2))
reg_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        padding="same"))
reg_model.add(k1.core.Activation("relu"))
reg_model.add(k1.Dropout(0.2))
reg_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        padding="same"))
reg_model.add(k1.core.Activation("relu"))
reg_model.add(k1.pooling.MaxPooling1D(pool_size=40,padding="same",
                                               strides=40))
reg_model.add(Flatten())
reg_model.add(k1.Dense(units = 100, activation = "relu"))
reg_model.add(k1.Dense(units = 1))

In [ ]:
reg_model.compile(optimizer="adam", loss="mean_squared_error")
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 60,
                              restore_best_weights=True)
history_reg = reg_model.fit_generator(generator=keras_train_batch_generator, 
                                      epochs=300, callbacks =[early_stopping_callback], 
                                      validation_data=keras_valid_batch_generator)
reg_model.set_weights(early_stopping_callback.best_weights)

Epoch 1/300
570/570 [==============================] - 30s 53ms/step - loss: 9011.0790 - val_loss: 10105.5901
Epoch 2/300
570/570 [==============================] - 33s 57ms/step - loss: 8658.8558 - val_loss: 10115.7214
Epoch 3/300
570/570 [==============================] - 29s 51ms/step - loss: 8614.1054 - val_loss: 10610.8356
Epoch 4/300
570/570 [==============================] - 31s 54ms/step - loss: 8606.6016 - val_loss: 10597.5397
Epoch 5/300
570/570 [==============================] - 30s 52ms/step - loss: 8539.7052 - val_loss: 10013.3539
Epoch 6/300
570/570 [==============================] - 29s 51ms/step - loss: 7939.1870 - val_loss: 8555.4137
Epoch 7/300
570/570 [==============================] - 31s 54ms/step - loss: 7214.2861 - val_loss: 7590.5500
Epoch 8/300
570/570 [==============================] - 29s 50ms/step - loss: 6899.4491 - val_loss: 7329.7668
Epoch 9/300
570/570 [==============================] - 30s 52ms/step - loss: 6693.6004 - val_loss: 6952.7731
Epoch 10/300
 

In [ ]:
reg_filename = ('reg_dropout_%s.h5' % seed_num, str(seed_num))[0]
reg_model.save(reg_filename)
reg_model_final = load_model(reg_filename)

In [ ]:
augment_model = keras.models.Sequential()
augment_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        input_shape=keras_train_batch_generator_augment[0][0].shape[1:], padding="same"))
augment_model.add(k1.core.Activation("relu"))
augment_model.add(k1.Dropout(0.2))
augment_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        padding="same"))
augment_model.add(k1.core.Activation("relu"))
augment_model.add(k1.Dropout(0.2))
augment_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        padding="same"))
augment_model.add(k1.core.Activation("relu"))
augment_model.add(k1.pooling.MaxPooling1D(pool_size=40,padding="same",
                                               strides=40))
augment_model.add(Flatten())
augment_model.add(k1.Dense(units = 100, activation = "relu"))
augment_model.add(k1.Dense(units = 1))

In [ ]:
augment_model.compile(optimizer="adam", loss="mean_squared_error")
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 60,
                              restore_best_weights=True)
history_aug = augment_model.fit_generator(generator=keras_train_batch_generator_augment, 
                                      epochs=300, callbacks =[early_stopping_callback], 
                                      validation_data=keras_valid_batch_generator_augment)
augment_model.set_weights(early_stopping_callback.best_weights)
augment_filename = ('augment_dropout_%s.h5' % seed_num, str(seed_num))[0]
augment_model.save(augment_filename)

In [ ]:
augment_model_spatial_dropout = keras.models.Sequential()
augment_model_spatial_dropout.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        input_shape=keras_train_batch_generator_augment[0][0].shape[1:], padding="same"))
augment_model_spatial_dropout.add(k1.core.Activation("relu"))
augment_model_spatial_dropout.add(RevCompSpatialDropout1D(0.2))
augment_model_spatial_dropout.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        padding="same"))
augment_model_spatial_dropout.add(k1.core.Activation("relu"))
augment_model_spatial_dropout.add(RevCompSpatialDropout1D(0.2))
augment_model_spatial_dropout.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        padding="same"))
augment_model_spatial_dropout.add(k1.core.Activation("relu"))
augment_model_spatial_dropout.add(k1.pooling.MaxPooling1D(pool_size=40,padding="same",
                                               strides=40))
augment_model_spatial_dropout.add(Flatten())
augment_model_spatial_dropout.add(k1.Dense(units = 100, activation = "relu"))
augment_model_spatial_dropout.add(k1.Dense(units = 1))

In [ ]:
augment_model_spatial_dropout.compile(optimizer="adam", loss="mean_squared_error")
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 60,
                              restore_best_weights=True)
history_aug_spatial_dropout = augment_model_spatial_dropout.fit_generator(generator=keras_train_batch_generator_augment, 
                                      epochs=300, callbacks =[early_stopping_callback], 
                                      validation_data=keras_valid_batch_generator_augment)
augment_model_spatial_dropout.set_weights(early_stopping_callback.best_weights)
augment_filename = ('augment_spatial_dropout_%s.h5' % seed_num, str(seed_num))[0]
augment_spatial_dropout_model.save(augment_filename)

In [ ]:
rc_model_standard_mcdropout = keras.models.Sequential()
rc_model_standard_mcdropout.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, 
            input_shape=keras_train_batch_generator[0][0].shape[1:], padding="same"))
rc_model_standard_mcdropout.add(k1.core.Activation("relu"))
rc_model_standard_mcdropout.add(MCRCDropout(0.2))
rc_model_standard_mcdropout.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, padding="same"))
rc_model_standard_mcdropout.add(k1.core.Activation("relu"))
rc_model_standard_mcdropout.add(MCRCDropout(0.2))
rc_model_standard_mcdropout.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size,padding="same"))
rc_model_standard_mcdropout.add(k1.core.Activation("relu"))
rc_model_standard_mcdropout.add(RevCompSumPool())
rc_model_standard_mcdropout.add(k1.pooling.MaxPooling1D(pool_size=40,padding="same", strides=40))
rc_model_standard_mcdropout.add(Flatten())
rc_model_standard_mcdropout.add(keras_genomics.layers.core.Dense(units = 100, activation = "relu"))
rc_model_standard_mcdropout.add(keras_genomics.layers.core.Dense(units = 1))

In [ ]:
rc_model_standard_mcdropout.compile(optimizer="adam", loss='mean_squared_error')
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 60,
                              restore_best_weights=True)

history_rc_standard_mcdropout = rc_model_standard_mcdropout.fit_generator(generator=keras_train_batch_generator, 
                                                      epochs=300, callbacks = [early_stopping_callback],
                                                     validation_data=keras_valid_batch_generator)
    
rc_model_standard_mcdropout.set_weights(early_stopping_callback.best_weights)
rc_standard_filename_mcdropout = ('rc_standard_mcdropout_%s.h5' % seed_num, str(seed_num))[0]
rc_model_standard_mcdropout.save(rc_standard_filename_mcdropout)

In [ ]:
rc_model_standard_spatial_dropout = keras.models.Sequential()
rc_model_standard_spatial_dropout.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, 
            input_shape=keras_train_batch_generator[0][0].shape[1:], padding="same"))
rc_model_standard_spatial_dropout.add(k1.core.Activation("relu"))
rc_model_standard_spatial_dropout.add(RevCompSpatialDropout1D(0.2))
rc_model_standard_spatial_dropout.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, padding="same"))
rc_model_standard_spatial_dropout.add(k1.core.Activation("relu"))
rc_model_standard_spatial_dropout.add(RevCompSpatialDropout1D(0.2))
rc_model_standard_spatial_dropout.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size,padding="same"))
rc_model_standard_spatial_dropout.add(k1.core.Activation("relu"))
rc_model_standard_spatial_dropout.add(RevCompSumPool())
rc_model_standard_spatial_dropout.add(k1.pooling.MaxPooling1D(pool_size=40,padding="same", strides=40))
rc_model_standard_spatial_dropout.add(Flatten())
rc_model_standard_spatial_dropout.add(keras_genomics.layers.core.Dense(units = 100, activation = "relu"))
rc_model_standard_spatial_dropout.add(keras_genomics.layers.core.Dense(units = 1))

In [ ]:
rc_model_standard_spatial_dropout.compile(optimizer="adam",loss='mean_squared_error')
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 60,
                              restore_best_weights=True)
history_rc_spatial_dropout = rc_model_standard_spatial_dropout.fit_generator(generator=keras_train_batch_generator, 
                                                      epochs=300, callbacks = [early_stopping_callback],
                                                     validation_data=keras_valid_batch_generator)
    
rc_model_standard_spatial_dropout.set_weights(early_stopping_callback.best_weights)
rc_standard_filename_spatial_dropout = ('rc_standard_spatial_dropout_%s.h5' % seed_num, str(seed_num))[0]
rc_model_standard_spatial_dropout.save(rc_standard_filename_spatial_dropout)

In [ ]:
rc_model_standard_dropout = keras.models.Sequential()
rc_model_standard_dropout.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, 
            input_shape=keras_train_batch_generator[0][0].shape[1:], padding="same"))
rc_model_standard_dropout.add(k1.core.Activation("relu"))
rc_model_standard_dropout.add(k1.Dropout(0.2))
rc_model_standard_dropout.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, padding="same"))
rc_model_standard_dropout.add(k1.core.Activation("relu"))
rc_model_standard_dropout.add(k1.Dropout(0.2))
rc_model_standard_dropout.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size,padding="same"))
rc_model_standard_dropout.add(k1.core.Activation("relu"))
rc_model_standard_dropout.add(RevCompSumPool())
rc_model_standard_dropout.add(k1.pooling.MaxPooling1D(pool_size=40,padding="same", strides=40))
rc_model_standard_dropout.add(Flatten())
rc_model_standard_dropout.add(keras_genomics.layers.core.Dense(units = 100, activation = "relu"))
rc_model_standard_dropout.add(keras_genomics.layers.core.Dense(units = 1))

In [ ]:
rc_model_standard_dropout.compile(optimizer="adam",loss='mean_squared_error')
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 60,
                              restore_best_weights=True)
history_rc_dropout = rc_model_standard_dropout.fit_generator(generator=keras_train_batch_generator, 
                                                      epochs=300, callbacks = [early_stopping_callback],
                                                     validation_data=keras_valid_batch_generator)
    
rc_model_standard_dropout.set_weights(early_stopping_callback.best_weights)
rc_standard_filename_dropout = ('rc_standard_dropout_%s.h5' % seed_num, str(seed_num))[0]
rc_model_standard_dropout.save(rc_standard_filename_dropout)

In [ ]:
s_model = Sequential([
    k1.Conv1D(filters=filters, kernel_size=kernel_size,
            input_shape=keras_train_batch_generator[0][0].shape[1:], padding="same"), 
    k1.core.Activation("relu"),
    k1.Conv1D(filters=filters, kernel_size=kernel_size,
              padding="same"), 
    k1.core.Activation("relu"),
    k1.Conv1D(filters=filters, kernel_size=kernel_size,
              padding="same"), 
    k1.core.Activation("relu"),
    k1.pooling.MaxPooling1D(pool_size=40,padding="same",
                                               strides=40), 
    Flatten(), 
    k1.Dense(units = 100, activation = "relu"),
    k1.Dense(units = 1)
], name = "shared_layers")

main_input = Input(shape=keras_train_batch_generator[0][0].shape[1:])
rev_input = Input(shape=keras_train_batch_generator[0][0].shape[1:])

rev_input = RevComp()(main_input)

main_output = s_model(main_input)
rev_output = s_model(rev_input)

avg = k1.Average()([main_output, rev_output])
siamese_model = Model(inputs = main_input, outputs = avg)

merged = keras.layers.concatenate([main_output, rev_output])
                                  
siamese_model.compile(optimizer="adam", loss="mean_squared_error")
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 60,
                              restore_best_weights=True)
siamese_model.fit_generator(generator= keras_train_batch_generator, 
                           epochs=300, callbacks=[early_stopping_callback],
                           validation_data=keras_valid_batch_generator)
siamese_model.set_weights(early_stopping_callback.best_weights)  

siamese_filename = ('siamese_no_dropout_%s.h5' % seed_num, str(seed_num))[0]
siamese_model.save(siamese_filename)

In [ ]:
s_model_dropout = Sequential([
    k1.Conv1D(filters=filters, kernel_size=kernel_size,
            input_shape=keras_train_batch_generator[0][0].shape[1:], padding="same"), 
    k1.core.Activation("relu"),
    k1.Dropout(0.2),
    k1.Conv1D(filters=filters, kernel_size=kernel_size,
              padding="same"), 
    k1.core.Activation("relu"),
    k1.Dropout(0.2),
    k1.Conv1D(filters=filters, kernel_size=kernel_size,
              padding="same"), 
    k1.core.Activation("relu"),
    k1.pooling.MaxPooling1D(pool_size=40,padding="same",
                                               strides=40), 
    Flatten(), 
    k1.Dense(units = 100, activation = "relu"),
    k1.Dense(units = 1)
], name = "shared_layers")

main_input = Input(shape=keras_train_batch_generator[0][0].shape[1:])
rev_input = Input(shape=keras_train_batch_generator[0][0].shape[1:])

rev_input = RevComp()(main_input)

main_output = s_model_dropout(main_input)
rev_output = s_model_dropout(rev_input)

avg = k1.Average()([main_output, rev_output])
siamese_model_dropout = Model(inputs = main_input, outputs = avg)

merged = keras.layers.concatenate([main_output, rev_output])
                                  
siamese_model_dropout.compile(optimizer="adam", loss="mean_squared_error")
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 60,
                              restore_best_weights=True)
siamese_model_dropout.fit_generator(generator= keras_train_batch_generator, 
                           epochs=300, callbacks=[early_stopping_callback],
                           validation_data=keras_valid_batch_generator)
siamese_model_dropout.set_weights(early_stopping_callback.best_weights)  

siamese_filename = ('siamese_dropout_%s.h5' % seed_num, str(seed_num))[0]
siamese_model_dropout.save(siamese_filename)

In [ ]:
s_model_spatial_dropout = Sequential([
    k1.Conv1D(filters=filters, kernel_size=kernel_size,
            input_shape=keras_train_batch_generator[0][0].shape[1:], padding="same"), 
    k1.core.Activation("relu"),
    RevCompSpatialDropout1D(0.2), 
    k1.Conv1D(filters=filters, kernel_size=kernel_size,
              padding="same"), 
    k1.core.Activation("relu"),
    RevCompSpatialDropout1D(0.2), 
    k1.Conv1D(filters=filters, kernel_size=kernel_size,
              padding="same"), 
    k1.core.Activation("relu"),
    k1.pooling.MaxPooling1D(pool_size=40,padding="same",
                                               strides=40), 
    Flatten(), 
    k1.Dense(units = 100, activation = "relu"),
    k1.Dense(units = 1)
], name = "shared_layers")

main_input = Input(shape=keras_train_batch_generator[0][0].shape[1:])
rev_input = Input(shape=keras_train_batch_generator[0][0].shape[1:])

rev_input = RevComp()(main_input)

main_output = s_model_spatial_dropout(main_input)
rev_output = s_model_spatial_dropout(rev_input)

avg = k1.Average()([main_output, rev_output])
siamese_model_spatial_dropout = Model(inputs = main_input, outputs = avg)

merged = keras.layers.concatenate([main_output, rev_output])
                                  
siamese_model_spatial_dropout.compile(optimizer="adam", loss="mean_squared_error")
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 60,
                              restore_best_weights=True)
siamese_model_spatial_dropout.fit_generator(generator= keras_train_batch_generator, 
                           epochs=300, callbacks=[early_stopping_callback],
                           validation_data=keras_valid_batch_generator)
siamese_model_spatial_dropout.set_weights(early_stopping_callback.best_weights)  

siamese_filename = ('siamese_spatial_dropout_%s.h5' % seed_num, str(seed_num))[0]
siamese_model_spatial_dropout.save(siamese_filename)